In [0]:
!pip install transformers

In [0]:
!pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.2.3)


In [0]:
import torch
torch.cuda.is_available()

True

In [0]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
"""
This examples trains BERT for the STSbenchmark from scratch. It generates sentence embeddings
that can be compared using cosine-similarity to measure the similarity.
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, models
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import STSDataReader
import logging
from datetime import datetime


#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

In [0]:
!gdown https://drive.google.com/uc?id=1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD

Downloading...
From: https://drive.google.com/uc?id=1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD
To: /content/biobert_v1.1_pubmed.tar.gz
401MB [00:02, 172MB/s]


In [0]:
!tar xvzf biobert_v1.1_pubmed.tar.gz

biobert_v1.1_pubmed/
biobert_v1.1_pubmed/model.ckpt-1000000.data-00000-of-00001
biobert_v1.1_pubmed/model.ckpt-1000000.meta
biobert_v1.1_pubmed/bert_config.json
biobert_v1.1_pubmed/vocab.txt
biobert_v1.1_pubmed/model.ckpt-1000000.index


In [0]:
!transformers bert biobert_v1.1_pubmed/model.ckpt-1000000 biobert_v1.1_pubmed/bert_config.json biobert_v1.1_pubmed/pytorch_model.bin

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 28996
}

INFO:transformers.modeling_bert:Converting TensorFlow checkpoint from /content/biobert_v1.1_pubmed/model.ckpt-1000000
INFO:transformers.modeling_bert:Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/embeddings/LayerNorm/gamma with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/embeddings/position_

In [0]:
import gc
gc.collect()

20

In [0]:
#after this, renamed bert_config.json to config.json manually
!cp biobert_v1.1_pubmed/bert_config.json biobert_v1.1_pubmed/config.json

In [0]:
# Read the dataset
''' 16 and 4 used for bert_stsb, bert_sts_biomedical and biobert_sts_biomedcial'''
train_batch_size = 16
num_epochs = 4



model_save_path = 'output/training_n2c2_sts_bert-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# parameters for STSDataReader -> dataset_folder, s1_col_idx=5, s2_col_idx=6, score_col_idx=4, delimiter="\t", quoting=csv.QUOTE_NONE, normalize_scores=True, min_score=0, max_score=5
sts_reader = STSDataReader('', s1_col_idx=0, s2_col_idx=1, score_col_idx=2, normalize_scores=True)



'''# Map tokens to vectors using BERT
word_embedding_model = models.BERT('biobert_v1.1_pubmed') #('bert-base-uncased')

#cnn = models.CNN(in_word_embedding_dimension=word_embedding_model.get_word_embedding_dimension(), out_channels=256, kernel_sizes=[1,3,5])

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(cnn.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)


model = SentenceTransformer(modules=[word_embedding_model, cnn, pooling_model])'''

# Use BERT for mapping tokens to embeddings
word_embedding_model = models.BERT('biobert_v1.1_pubmed') #('bert-base-uncased')

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=True,
                               pooling_mode_max_tokens=True)

#dense1 = models.Dense(in_features=3*word_embedding_model.get_word_embedding_dimension(), out_features=128)
#dense2 = models.Dense(in_features=256, out_features=64)
#dense3 = models.Dense(in_features=64, out_features=8)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


Epoch:   0%|          | 0/3 [00:48<?, ?it/s]

Epoch:   0%|          | 0/3 [00:48<?, ?it/s]

Iteration:  24%|██▍       | 15/62 [00:48<00:40,  1.16it/s]

2019-12-04 09:18:58 - loading configuration file biobert_v1.1_pubmed/config.json
2019-12-04 09:18:58 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

2019-12-04 09:18:58 - loading weights file biobert_v1.1_pubmed/pytorch_model.bin



Epoch:   0%|          | 0/3 [00:51<?, ?it/s]

Epoch:   0%|          | 0/3 [00:51<?, ?it/s]

Epoch:   0%|          | 0/3 [00:51<?, ?it/s]

Epoch:   0%|          | 0/3 [00:51<?, ?it/s]

Epoch:   0%|          | 0/3 [00:51<?, ?it/s]

Iteration:  24%|██▍       | 15/62 [00:51<00:40,  1.16it/s]

2019-12-04 09:19:00 - Model name 'biobert_v1.1_pubmed' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc). Assuming 'biobert_v1.1_pubmed' is a path or url to a directory containing tokenizer files.
2019-12-04 09:19:00 - Didn't find file biobert_v1.1_pubmed/added_tokens.json. We won't load it.
2019-12-04 09:19:00 - Didn't find file biobert_v1.1_pubmed/special_tokens_map.json. We won't load it.
2019-12-04 09:19:00 - loading file biobert_v1.1_pubmed/vocab.txt
2019-12-04 09:19:00 - loading file None
2019-12-04 09:19:00 - loading file None



Iteration:  24%|██▍       | 15/62 [00:51<00:40,  1.16it/s]

2019-12-04 09:19:01 - Use pytorch device: cuda


In [0]:
#sts_reader = STSDataReader('', s1_col_idx=0, s2_col_idx=1, score_col_idx=2, normalize_scores=True)

# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark train dataset")
train_data = SentencesDataset(sts_reader.get_examples('clinicalSTS2019.train_train60.csv'), model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


logging.info("Read STSbenchmark dev dataset")
dev_data = SentencesDataset(examples=sts_reader.get_examples('clinicalSTS2019.train_validate20.csv'), model=model)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=train_batch_size)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)



Iteration:  24%|██▍       | 15/62 [00:51<00:40,  1.16it/s]

Convert dataset:   0%|          | 0/992 [00:00<?, ?it/s]

2019-12-04 09:19:01 - Read STSbenchmark train dataset




Convert dataset:   9%|▉         | 89/992 [00:00<00:01, 887.55it/s]

Convert dataset:  17%|█▋        | 172/992 [00:00<00:00, 869.40it/s]

Convert dataset:  26%|██▌       | 254/992 [00:00<00:00, 850.71it/s]

Convert dataset:  34%|███▍      | 339/992 [00:00<00:00, 847.31it/s]

Convert dataset:  43%|████▎     | 425/992 [00:00<00:00, 849.85it/s]

Convert dataset:  52%|█████▏    | 513/992 [00:00<00:00, 857.04it/s]

Convert dataset:  60%|██████    | 600/992 [00:00<00:00, 860.54it/s]

Convert dataset:  68%|██████▊   | 679/992 [00:00<00:00, 814.87it/s]

Convert dataset:  77%|███████▋  | 762/992 [00:00<00:00, 817.35it/s]

Convert dataset:  86%|████████▌ | 855/992 [00:01<00:00, 847.38it/s]

Convert dataset:  96%|█████████▌| 949/992 [00:01<00:00, 872.96it/s]


Epoch:   0%|          | 0/3 [00:52<?, ?it/s]

Epoch:   0%|          | 0/3 [00:52<?, ?it/s]

Epoch:   0%|          | 0/3 [00:52<?, ?it/s]

Iteration:  24%|██▍       | 15/62 [00:52<00:40,  1.16it/s]

Convert dataset:   0%|          | 0/331 [

2019-12-04 09:19:02 - Num sentences: 992
2019-12-04 09:19:02 - Sentences 0 longer than max_seqence_length: 0
2019-12-04 09:19:02 - Sentences 1 longer than max_seqence_length: 0
2019-12-04 09:19:02 - Read STSbenchmark dev dataset




Convert dataset:  38%|███▊      | 125/331 [00:00<00:00, 514.41it/s]

Convert dataset:  66%|██████▌   | 219/331 [00:00<00:00, 594.84it/s]

Convert dataset:  93%|█████████▎| 307/331 [00:00<00:00, 657.93it/s]


Epoch:   0%|          | 0/3 [00:53<?, ?it/s]

Epoch:   0%|          | 0/3 [00:53<?, ?it/s]

Iteration:  24%|██▍       | 15/62 [00:53<00:40,  1.16it/s]

2019-12-04 09:19:02 - Num sentences: 331
2019-12-04 09:19:02 - Sentences 0 longer than max_seqence_length: 0
2019-12-04 09:19:02 - Sentences 1 longer than max_seqence_length: 0


In [0]:
# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_data)*num_epochs/train_batch_size*0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))



Iteration:  24%|██▍       | 15/62 [00:53<00:40,  1.16it/s]

2019-12-04 09:19:02 - Warmup-steps: 19


In [0]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)



Epoch:   0%|          | 0/3 [00:00<?, ?it/s]


Iteration:   0%|          | 0/62 [00:00<?, ?it/s]


Iteration:   2%|▏         | 1/62 [00:01<01:11,  1.17s/it]


Iteration:   3%|▎         | 2/62 [00:01<01:01,  1.03s/it]


Iteration:   5%|▍         | 3/62 [00:02<00:59,  1.01s/it]


Iteration:   6%|▋         | 4/62 [00:03<01:00,  1.04s/it]


Iteration:   8%|▊         | 5/62 [00:04<00:55,  1.03it/s]


Iteration:  10%|▉         | 6/62 [00:06<00:59,  1.06s/it]


Iteration:  11%|█▏        | 7/62 [00:07<01:07,  1.22s/it]


Iteration:  13%|█▎        | 8/62 [00:08<00:57,  1.06s/it]


Iteration:  15%|█▍        | 9/62 [00:09<00:52,  1.02it/s]


Iteration:  16%|█▌        | 10/62 [00:10<00:54,  1.05s/it]


Iteration:  18%|█▊        | 11/62 [00:11<00:51,  1.00s/it]


Iteration:  19%|█▉        | 12/62 [00:11<00:45,  1.10it/s]


Iteration:  21%|██        | 13/62 [00:12<00:40,  1.20it/s]


Iteration:  23%|██▎       | 14/62 [00:13<00:44,  1.07it/s]


Iteration:  24%|██▍       | 15/62 [00:15<00:49,  1.05s

2019-12-04 09:20:06 - Evaluation the model on  dataset after epoch 0:





Convert Evaluating:   5%|▍         | 1/21 [00:00<00:04,  4.16it/s]


Convert Evaluating:  10%|▉         | 2/21 [00:00<00:06,  3.14it/s]


Convert Evaluating:  14%|█▍        | 3/21 [00:01<00:06,  2.69it/s]


Convert Evaluating:  19%|█▉        | 4/21 [00:01<00:06,  2.72it/s]


Convert Evaluating:  24%|██▍       | 5/21 [00:01<00:05,  3.15it/s]


Convert Evaluating:  29%|██▊       | 6/21 [00:02<00:04,  3.09it/s]


Convert Evaluating:  33%|███▎      | 7/21 [00:02<00:05,  2.56it/s]


Convert Evaluating:  38%|███▊      | 8/21 [00:03<00:05,  2.52it/s]


Convert Evaluating:  43%|████▎     | 9/21 [00:03<00:04,  2.97it/s]


Convert Evaluating:  48%|████▊     | 10/21 [00:03<00:03,  2.95it/s]


Convert Evaluating:  52%|█████▏    | 11/21 [00:03<00:03,  3.19it/s]


Convert Evaluating:  57%|█████▋    | 12/21 [00:04<00:02,  3.18it/s]


Convert Evaluating:  62%|██████▏   | 13/21 [00:04<00:02,  3.04it/s]


Convert Evaluating:  67%|██████▋   | 14/21 [00:04<00:02,  3.14it/s]


Convert Evaluating:  71%|█

2019-12-04 09:20:13 - Cosine-Similarity :	Pearson: 0.7412	Spearman: 0.7425
2019-12-04 09:20:13 - Manhattan-Distance:	Pearson: 0.7138	Spearman: 0.7346
2019-12-04 09:20:13 - Euclidean-Distance:	Pearson: 0.7154	Spearman: 0.7352
2019-12-04 09:20:13 - Dot-Product-Similarity:	Pearson: 0.7419	Spearman: 0.7379
2019-12-04 09:20:13 - Save model to output/training_n2c2_sts_bert-2019-12-04_09-18-58




Epoch:  33%|███▎      | 1/3 [01:11<02:23, 71.84s/it]








Epoch:   0%|          | 0/3 [02:05<?, ?it/s]

Iteration:  24%|██▍       | 15/62 [02:05<00:40,  1.16it/s]


Iteration:   0%|          | 0/62 [00:00<?, ?it/s]

...




Iteration:   2%|▏         | 1/62 [00:00<00:45,  1.34it/s]


Iteration:   3%|▎         | 2/62 [00:01<00:44,  1.34it/s]


Iteration:   5%|▍         | 3/62 [00:02<00:44,  1.31it/s]


Iteration:   6%|▋         | 4/62 [00:03<00:47,  1.23it/s]


Iteration:   8%|▊         | 5/62 [00:04<00:57,  1.00s/it]


Iteration:  10%|▉         | 6/62 [00:05<00:52,  1.07it/s]


Iteration:  11%|█▏        | 7/62 [00:06<00:55,  1.01s/it]


Iteration:  13%|█▎        | 8/62 [00:07<00:53,  1.00it/s]


Iteration:  15%|█▍        | 9/62 [00:08<00:51,  1.02it/s]


Iteration:  16%|█▌        | 10/62 [00:09<00:53,  1.02s/it]


Iteration:  18%|█▊        | 11/62 [00:10<00:50,  1.00it/s]


Iteration:  19%|█▉        | 12/62 [00:11<00:49,  1.01it/s]


Iteration:  21%|██        | 13/62 [00:12<00:45,  1.08it/s]


Iteration:  23%|██▎       | 14/62 [00:13<00:44,  1.09it/s]


Iteration:  24%|██▍       | 15/62 [00:14<00:43,  1.08it/s]


Iteration:  26%|██▌       | 16/62 [00:15<00:49,  1.07s/it]


Iteration:  27%|██▋       | 17

2019-12-04 09:22:32 - Evaluation the model on  dataset after epoch 2:





Convert Evaluating:   5%|▍         | 1/21 [00:00<00:04,  4.12it/s]


Convert Evaluating:  10%|▉         | 2/21 [00:00<00:06,  3.13it/s]


Convert Evaluating:  14%|█▍        | 3/21 [00:01<00:06,  2.69it/s]


Convert Evaluating:  19%|█▉        | 4/21 [00:01<00:06,  2.74it/s]


Convert Evaluating:  24%|██▍       | 5/21 [00:01<00:05,  3.17it/s]


Convert Evaluating:  29%|██▊       | 6/21 [00:02<00:04,  3.10it/s]


Convert Evaluating:  33%|███▎      | 7/21 [00:02<00:05,  2.56it/s]


Convert Evaluating:  38%|███▊      | 8/21 [00:03<00:05,  2.52it/s]


Convert Evaluating:  43%|████▎     | 9/21 [00:03<00:04,  2.97it/s]


Convert Evaluating:  48%|████▊     | 10/21 [00:03<00:03,  2.95it/s]


Convert Evaluating:  52%|█████▏    | 11/21 [00:03<00:03,  3.20it/s]


Convert Evaluating:  57%|█████▋    | 12/21 [00:04<00:02,  3.18it/s]


Convert Evaluating:  62%|██████▏   | 13/21 [00:04<00:02,  3.04it/s]


Convert Evaluating:  67%|██████▋   | 14/21 [00:04<00:02,  3.15it/s]


Convert Evaluating:  71%|█

2019-12-04 09:22:40 - Cosine-Similarity :	Pearson: 0.7708	Spearman: 0.7736
2019-12-04 09:22:40 - Manhattan-Distance:	Pearson: 0.7447	Spearman: 0.7678
2019-12-04 09:22:40 - Euclidean-Distance:	Pearson: 0.7443	Spearman: 0.7678
2019-12-04 09:22:40 - Dot-Product-Similarity:	Pearson: 0.7724	Spearman: 0.7736
2019-12-04 09:22:40 - Save model to output/training_n2c2_sts_bert-2019-12-04_09-18-58




Epoch: 100%|██████████| 3/3 [03:38<00:00, 72.51s/it]



In [0]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

model = SentenceTransformer(model_save_path)
test_data = SentencesDataset(examples=sts_reader.get_examples("clinicalSTS2019.train_test20.csv"), model=model)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=train_batch_size)
evaluator = EmbeddingSimilarityEvaluator(test_dataloader)
model.evaluate(evaluator)


Epoch:   0%|          | 0/3 [04:31<?, ?it/s]

Epoch:   0%|          | 0/3 [04:31<?, ?it/s]

Epoch:   0%|          | 0/3 [04:31<?, ?it/s]

Epoch:   0%|          | 0/3 [04:31<?, ?it/s]

Iteration:  24%|██▍       | 15/62 [04:31<00:40,  1.16it/s]

2019-12-04 09:22:41 - Load pretrained SentenceTransformer: output/training_n2c2_sts_bert-2019-12-04_09-18-58
2019-12-04 09:22:41 - Load SentenceTransformer from folder: output/training_n2c2_sts_bert-2019-12-04_09-18-58
2019-12-04 09:22:41 - loading configuration file output/training_n2c2_sts_bert-2019-12-04_09-18-58/0_BERT/config.json
2019-12-04 09:22:41 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

2019-12-04 09:22:41 - loading weights file output/training_n2c2_sts_bert-2019-12-04_09-18-58/0_BERT/pytorch_model.bin



Epoch:   0%|          | 0/3 [04:33<?, ?it/s]

Epoch:   0%|          | 0/3 [04:33<?, ?it/s]

Epoch:   0%|          | 0/3 [04:33<?, ?it/s]

Epoch:   0%|          | 0/3 [04:33<?, ?it/s]



2019-12-04 09:22:43 - Model name 'output/training_n2c2_sts_bert-2019-12-04_09-18-58/0_BERT' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc). Assuming 'output/training_n2c2_sts_bert-2019-12-04_09-18-58/0_BERT' is a path or url to a directory containing tokenizer files.
2019-12-04 09:22:43 - loading file output/training_n2c2_sts_bert-2019-12-04_09-18-58/0_BERT/vocab.txt
2019-12-04 09:22:43 - loading file output/training_n2c2_sts_bert-2019-12-04_09-18-58/0_BERT/added_tokens.json
2019-12-04 09:22:43 - loading file output/training_n2c2_sts_bert-2019-12-04_09-18-58/0_BERT/special_tokens_map.json


Iteration:  24%|██▍       | 15/62 [04:34<00:40,  1.16it/s]

Convert dataset:   0%|          | 0/331 [00:00<?, ?it/s]

...



Convert dataset:  28%|██▊       | 94/331 [00:00<00:00, 939.36it/s]

Convert dataset:  58%|█████▊    | 191/331 [00:00<00:00, 945.76it/s]

Convert dataset:  86%|████████▌ | 285/331 [00:00<00:00, 942.04it/s]


Epoch:   0%|          | 0/3 [04:34<?, ?it/s]

Epoch:   0%|          | 0/3 [04:34<?, ?it/s]

Epoch:   0%|          | 0/3 [04:34<?, ?it/s]

Iteration:  24%|██▍       | 15/62 [04:34<00:40,  1.16it/s]

Convert Evaluating:   0%|          | 0/21 [00:00<?, ?it/s]

2019-12-04 09:22:44 - Num sentences: 331
2019-12-04 09:22:44 - Sentences 0 longer than max_seqence_length: 0
2019-12-04 09:22:44 - Sentences 1 longer than max_seqence_length: 0
2019-12-04 09:22:44 - Evaluation the model on  dataset:




Convert Evaluating:   5%|▍         | 1/21 [00:00<00:05,  3.56it/s]

Convert Evaluating:  10%|▉         | 2/21 [00:00<00:06,  3.09it/s]

Convert Evaluating:  14%|█▍        | 3/21 [00:01<00:06,  2.94it/s]

Convert Evaluating:  19%|█▉        | 4/21 [00:01<00:05,  3.32it/s]

Convert Evaluating:  24%|██▍       | 5/21 [00:01<00:05,  2.81it/s]

Convert Evaluating:  29%|██▊       | 6/21 [00:02<00:04,  3.07it/s]

Convert Evaluating:  33%|███▎      | 7/21 [00:02<00:04,  2.93it/s]

Convert Evaluating:  38%|███▊      | 8/21 [00:02<00:04,  2.83it/s]

Convert Evaluating:  43%|████▎     | 9/21 [00:02<00:03,  3.23it/s]

Convert Evaluating:  48%|████▊     | 10/21 [00:03<00:03,  2.94it/s]

Convert Evaluating:  52%|█████▏    | 11/21 [00:03<00:03,  2.80it/s]

Convert Evaluating:  57%|█████▋    | 12/21 [00:04<00:03,  2.95it/s]

Convert Evaluating:  62%|██████▏   | 13/21 [00:04<00:02,  3.06it/s]

Convert Evaluating:  67%|██████▋   | 14/21 [00:04<00:02,  3.38it/s]

Convert Evaluating:  71%|███████▏  | 15/2

2019-12-04 09:22:50 - Cosine-Similarity :	Pearson: 0.7379	Spearman: 0.7280
2019-12-04 09:22:50 - Manhattan-Distance:	Pearson: 0.7124	Spearman: 0.7284
2019-12-04 09:22:50 - Euclidean-Distance:	Pearson: 0.7098	Spearman: 0.7279
2019-12-04 09:22:50 - Dot-Product-Similarity:	Pearson: 0.7347	Spearman: 0.7192


0.7284098353176021

In [0]:
!zip -r output.zip output

updating: output/ (stored 0%)
updating: output/training_n2c2_sts_bert-2019-12-04_08-34-47/ (stored 0%)
updating: output/training_n2c2_sts_bert-2019-12-04_08-34-47/2_Dense/ (stored 0%)
updating: output/training_n2c2_sts_bert-2019-12-04_08-34-47/2_Dense/pytorch_model.bin (deflated 8%)
updating: output/training_n2c2_sts_bert-2019-12-04_08-34-47/2_Dense/config.json (deflated 22%)
updating: output/training_n2c2_sts_bert-2019-12-04_08-34-47/similarity_evaluation_results.csv (deflated 49%)
updating: output/training_n2c2_sts_bert-2019-12-04_08-34-47/3_Dense/ (stored 0%)
updating: output/training_n2c2_sts_bert-2019-12-04_08-34-47/3_Dense/pytorch_model.bin (deflated 8%)
updating: output/training_n2c2_sts_bert-2019-12-04_08-34-47/3_Dense/config.json (deflated 22%)
updating: output/training_n2c2_sts_bert-2019-12-04_08-34-47/config.json (stored 0%)
updating: output/training_n2c2_sts_bert-2019-12-04_08-34-47/modules.json (deflated 73%)
updating: output/training_n2c2_sts_bert-2019-12-04_08-34-47/1_Po

In [0]:
!cp output.zip '/content/gdrive/My Drive/biobert_finetuned_sts_biomedical_mean_only.zip'

KeyboardInterrupt: ignored